# Jobsheet 9 - Ensemble Method #

<b>IDENTITAS</b>

NIM : 2041720099

Nama : Rofika Nur 'Aini

Kelas : TI-3B

<b>Bagging</b>

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import RandomForestClassifier # import RandomForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# Load data
df = pd.read_csv('data/mushrooms.csv')

df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [3]:
# Cek kolom null
df.isnull().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [4]:
# Seleksi fitur

# Slice dataframe mulai dari kolom 'radius_mean' sampai 'fractal_dimension_worst'
x = df.iloc[:,1:]
y = df.iloc[:,0]

# Encode Fitur
from sklearn.preprocessing import LabelEncoder

def feature_encode(df):
    encode = LabelEncoder()
    for col in df:
        df[col] = encode.fit_transform(df[col])
    
    return df

x = feature_encode(x)
display(x)

# Encode Label
encode = LabelEncoder()
y = encode.fit_transform(y)
print(y)

# Cek jumlah fitur dan instance
x.shape

C:\Users\PAVILION\AppData\Local\Temp/ipykernel_2740/3488976834.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = encode.fit_transform(df[col])


,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,5,2,4,1,6,1,0,1,4,0,...,2,7,7,0,2,1,4,2,3,5
1,5,2,9,1,0,1,0,0,4,0,...,2,7,7,0,2,1,4,3,2,1
2,0,2,8,1,3,1,0,0,5,0,...,2,7,7,0,2,1,4,3,2,3
3,5,3,8,1,6,1,0,1,5,0,...,2,7,7,0,2,1,4,2,3,5
4,5,2,3,0,5,1,1,0,4,1,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,3,2,4,0,5,0,0,0,11,0,...,2,5,5,0,1,1,4,0,1,2
8120,5,2,4,0,5,0,0,0,11,0,...,2,5,5,0,0,1,4,0,4,2
8121,2,2,4,0,5,0,0,0,5,0,...,2,5,5,0,1,1,4,0,1,2
8122,3,3,4,0,8,1,0,1,0,1,...,1,7,7,0,2,1,0,7,4,2


[1 0 0 ... 0 1 0]


(8124, 22)

Split Data Training dan Testing

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

Training Decision Tree

In [6]:
# Secara default, DecisionTreeClassifier dari scikit-learn akan menggunakan nilai "Gini" untuk kriteria
# Terdapat beberapa "hyperparamater" yang dapat digunakan. Silahka baca dokumentasi
# Pada kasus ini kita akan menggunakan parameter default
dt = DecisionTreeClassifier()

# Sesuaikan dt ke set training
dt.fit(x_train, y_train)

# Memprediksi label set test
y_pred_dt = dt.predict(x_test)

#  menghitung set accuracy
acc_dt = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"Test set accuracy: {acc_dt}")

Test set accuracy: 1.00
Test set accuracy: 1.0


Training RandomForest

In [7]:
# Pada kasus kali ini kita akan menggunakan estimator pada RandomForest

rf = RandomForestClassifier(n_estimators=10) 
#(n_estimators) number of estimator berapa jumlah pohon yang mau dibuat atau berapa kali data akan di split
# untuk mengetahui estimator terbaik menggunakan GridSearchCV
# Sesuaikan dt ke set training
rf.fit(x_train, y_train)

# Memprediksi label set test
y_pred_rf = rf.predict(x_test)

#  menghitung set accuracy
acc_rf = accuracy_score(y_test, y_pred_rf)
print("Test set accuracy: {:.2f}".format(acc_rf))
print(f"Test set accuracy: {acc_rf}")

Test set accuracy: 1.00
Test set accuracy: 1.0


training data mushrooms menggunakan model decision tree dan random forest menghasilkan hasil akurasi tes yang sama yakni 1,0 atau 100%.

<b>Boosting</b>

In [8]:
from sklearn.ensemble import AdaBoostClassifier # import AdaBoost

# Pada kasus kali ini kita akan menggunakan estimator pada AdaBoost
ada = AdaBoostClassifier(n_estimators=10)
# Sesuaikan dt ke set training
ada.fit(x_train, y_train)

# Memprediksi label set test
y_pred_ada = ada.predict(x_test)

#  menghitung set accuracy
acc_ada = accuracy_score(y_test, y_pred_ada)
print("Test set accuracy: {:.2f}".format(acc_ada))
print(f"Test set accuracy: {acc_ada}")

Test set accuracy: 0.96
Test set accuracy: 0.9643076923076923


In [9]:
from sklearn.model_selection import GridSearchCV

# defining the classifier
model = AdaBoostClassifier()
# creating a dic for the grid
grid = dict()
# estimator till 500
grid['n_estimators'] = [10, 50]
# defining learning rate
grid['learning_rate'] = [0.0001, 0.01, 0.1, 1.0, 1.1, 1.2]
# initializing the grid search
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, scoring='accuracy')
# training the model on grid search for hyperparameter tuning of Adaboost
grid_result = grid_search.fit(x_train, y_train)
# finding the best results/hyperparameter tuning of Adaboost
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 1.000000 using {'learning_rate': 1.0, 'n_estimators': 50}


In [10]:
ada = AdaBoostClassifier(n_estimators=50)
# Sesuaikan dt ke set training
ada.fit(x_train, y_train)

# Memprediksi label set test
y_pred_ada = ada.predict(x_test)

#  menghitung set accuracy
acc_ada = accuracy_score(y_test, y_pred_ada)
print("Test set accuracy: {:.2f}".format(acc_ada))
print(f"Test set accuracy: {acc_ada}")

Test set accuracy: 1.00
Test set accuracy: 1.0


Awal training data mushrooms menggunakan model decision tree dan Ada Boost menghasilkan hasil akurasi tes yang berbeda yakni 100% dan 96%. Kemudian, mencari jumlah estimator terbaik menggunakan GridSearchCV yang menghasilkan estimator terbaik yakni 50. Lalu, training data dengan Ada Boost yang estimator nya 50 menghasilkan hasil tes akurasi 100%.

<b>Stacking</b>

In [11]:
# Load data
dft = pd.read_csv('data/diabetes.csv')
dft.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [12]:
# Cek kolom null
dft.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [13]:
# Cek kolom neng nilai 0
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing zeros : {len(dft.loc[dft[column] == 0])}")

Pregnancies ==> Missing zeros : 111
Glucose ==> Missing zeros : 5
BloodPressure ==> Missing zeros : 35
SkinThickness ==> Missing zeros : 227
Insulin ==> Missing zeros : 374
BMI ==> Missing zeros : 11
DiabetesPedigreeFunction ==> Missing zeros : 0
Age ==> Missing zeros : 0


In [14]:
# Impute nilai 0 dengan mean
from sklearn.impute import SimpleImputer

fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)

dft[feature_columns] = fill_values.fit_transform(dft[feature_columns])

In [15]:
#split data training dam test
xt = dft[feature_columns]
yt = dft.Outcome

xt_train, xt_test, yt_train, yt_test = train_test_split(xt, yt, test_size=0.3, random_state=42)

<b>Training</b>
Decision Tree

In [16]:
dt2 = DecisionTreeClassifier()

# Sesuaikan dt ke set training
dt2.fit(xt_train, yt_train)

# Memprediksi label set test
yt_pred_dt = dt2.predict(xt_test)

#  menghitung set accuracy
acc_dt2 = accuracy_score(yt_test, yt_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt2))
print(f"Test set accuracy: {acc_dt2}")

Test set accuracy: 0.71
Test set accuracy: 0.7056277056277056


tunning hyperparameter

In [17]:
for max_d in range(1,15):
  model2 = DecisionTreeClassifier(max_depth=max_d, random_state=42)
  model2.fit(xt_train, yt_train)
  yt_pred_dt2 = model2.predict(xt_test)
  acc = accuracy_score(yt_test, yt_pred_dt2)
  print('The Training Accuracy for max_depth {} is:'.format(max_d), model2.score(xt_train, yt_train))
  print('The Test Accuracy for max_depth {} is:'.format(max_d), acc)
  print('')

The Training Accuracy for max_depth 1 is: 0.7597765363128491
The Test Accuracy for max_depth 1 is: 0.7186147186147186

The Training Accuracy for max_depth 2 is: 0.7597765363128491
The Test Accuracy for max_depth 2 is: 0.7186147186147186

The Training Accuracy for max_depth 3 is: 0.7653631284916201
The Test Accuracy for max_depth 3 is: 0.7186147186147186

The Training Accuracy for max_depth 4 is: 0.8007448789571695
The Test Accuracy for max_depth 4 is: 0.70995670995671

The Training Accuracy for max_depth 5 is: 0.8249534450651769
The Test Accuracy for max_depth 5 is: 0.7359307359307359

The Training Accuracy for max_depth 6 is: 0.845437616387337
The Test Accuracy for max_depth 6 is: 0.6883116883116883

The Training Accuracy for max_depth 7 is: 0.8864059590316573
The Test Accuracy for max_depth 7 is: 0.6796536796536796

The Training Accuracy for max_depth 8 is: 0.9124767225325885
The Test Accuracy for max_depth 8 is: 0.670995670995671

The Training Accuracy for max_depth 9 is: 0.93482309

In [18]:
dt2 = DecisionTreeClassifier(max_depth=1)

# Sesuaikan dt ke set training
dt2.fit(xt_train, yt_train)

# Memprediksi label set test
yt_pred_dt = dt2.predict(xt_test)

#  menghitung set accuracy
acc_dt2 = accuracy_score(yt_test, yt_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt2))
print(f"Test set accuracy: {acc_dt2}")

Test set accuracy: 0.72
Test set accuracy: 0.7186147186147186


In [19]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(xt_train, yt_train)

#prediction
y_pred_lr = logreg.predict(xt_test)

# Evaluate test data accuracy
acc_lr = accuracy_score(yt_test, y_pred_lr)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_lr))
print(f"Test set accuracy: {acc_lr}")

Test set accuracy: 0.74
Test set accuracy: 0.7359307359307359


c:\Users\PAVILION\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
from sklearn.svm import SVC # import SVM classifier

# Model SVM Polynomial
svm_poly = SVC(kernel='poly', degree=8)

# Fit ke model
svm_poly.fit(xt_train, yt_train)

# Prediksi
y_pred_svm_poly = svm_poly.predict(xt_test)

# Evaluasi akurasi testing data
acc_svm_poly = accuracy_score(yt_test, y_pred_svm_poly)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_svm_poly))
print(f"Test set accuracy: {acc_svm_poly}")

Test set accuracy: 0.75
Test set accuracy: 0.7532467532467533


In [21]:
from sklearn.ensemble import VotingClassifier
# Definisikan algoritma yang akan digunakan untuk voting

clf1 = DecisionTreeClassifier()
clf2 = LogisticRegression()
clf3 = SVC(kernel='poly', probability=True)

# model hard voting
voting = VotingClassifier(estimators=[('DecisionTreeClassifier', clf1), ('LogisticRegression', clf2), ('SVM-POLY', clf3)], voting='hard')

# Fit model
voting.fit(xt_train, yt_train)

# Prediksi
y_pred_vt2 = voting.predict(xt_test)

# Evaluasi akurasi testing data
acc_vt2 = accuracy_score(yt_test, y_pred_vt2)

# Print hasil evaluasi
print('Voting Hard')
print("Test set accuracy: {:.2f}".format(acc_vt2))
print(f"Test set accuracy: {acc_vt2}")

Voting Hard
Test set accuracy: 0.74
Test set accuracy: 0.7445887445887446


c:\Users\PAVILION\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
